#### Notes:
1. Python ASN.1 tools library does not yet support parameterization, meaning DATE-TIME-FRACTIONAL isn't accepted. Test examples without it for now.
2. Schema is compiled from file, but message to be encoded currently must be input as a python dict.

In [1]:
import numpy as np
from datetime import datetime
import asn1tools
import asn1
import pyasn1
#import asn1vnparser

## Compile schema from file

In [2]:
schema_path = '/Users/michaelbondin/ASN1-Rad-Data-Streaming-Standard/standard_schema_complete_without_date-time-fractional.asn'
encoding_rule = 'der'
spec = asn1tools.compile_files(schema_path, encoding_rule)

## Provide message as python dict

#### Example Gross Gamma Counts Radiation Data Packet

In [60]:
message = {
    'radiationDataPacketNumber'   : 1,
    'timeInformationRadiationData' :
    {
        'relativeTime'             : 116011,
        'accumulationTime'         : 112911
    },
    'detectorsListRadiationData'   : [
        {
            'detectorID'           : 0,
            'counts'               : 273058,
            'channelSpectrum'      : [587, 9339, 13919, 6590, 6821, 7008, 7366, 7930, 
                                    8137, 8014, 7964, 9313, 10812, 9506, 8446, 7484, 
                                    6757, 6314, 5971, 5680, 5363, 5243, 5246, 5135, 
                                    5099, 5166, 5433, 5577, 5703, 4835, 2862, 1793, 
                                    1306, 900, 728, 615, 477, 510, 787, 3366, 14696, 
                                    24910, 9902, 935, 214, 191, 161, 157, 171, 189, 
                                    160, 169, 135, 144, 118, 115, 111, 110, 91, 89, 
                                    86, 64, 38, 587, 9339, 13919, 6590, 6821, 7008, 7366, 7930, 
                                    8137, 8014, 7964, 9313, 10812, 9506, 8446, 7484, 
                                    6757, 6314, 5971, 5680, 5363, 5243, 5246, 5135, 
                                    5099, 5166, 5433, 5577, 5703, 4835, 2862, 1793, 
                                    1306, 900, 728, 615, 477, 510, 787, 3366, 14696, 
                                    24910, 9902, 935, 214, 191, 161, 157, 171, 189, 
                                    160, 169, 135, 144, 118, 115, 111, 110, 91, 89, 
                                    86, 64, 38],
            'derivedData'          :
            {
                'countRate'        : 2744.299263,
                'deadTime'         : 6.375924139,
            }
        }
    ]
}

# message = {
#     'radiationDataPacketNumber'    : 3,
#     'timeInformationRadiationData' : {'relativeTime'     : 3010,
#                                       'accumulationTime' : 2000},
#     'detectorsListRadiationData'   : [{'detectorID'  : 0,
#                                        'counts'      : 6489,
#                                        'derivedData' : {'countRate'    : 3244.5,
#                                                         'exposure'     : 40.056,
#                                                         'exposureRate' : 72100.0,
#                                                         'alarm1'       : True,
#                                                         'deadTime'     : 10.0,
#                                                         'liveTime'     : 1800.0}
#                                        }],
#     'endRadiationDataPacket'       : 1110010
# }

message

{'radiationDataPacketNumber': 1,
 'timeInformationRadiationData': {'relativeTime': 116011,
  'accumulationTime': 112911},
 'detectorsListRadiationData': [{'detectorID': 0,
   'counts': 273058,
   'channelSpectrum': [587,
    9339,
    13919,
    6590,
    6821,
    7008,
    7366,
    7930,
    8137,
    8014,
    7964,
    9313,
    10812,
    9506,
    8446,
    7484,
    6757,
    6314,
    5971,
    5680,
    5363,
    5243,
    5246,
    5135,
    5099,
    5166,
    5433,
    5577,
    5703,
    4835,
    2862,
    1793,
    1306,
    900,
    728,
    615,
    477,
    510,
    787,
    3366,
    14696,
    24910,
    9902,
    935,
    214,
    191,
    161,
    157,
    171,
    189,
    160,
    169,
    135,
    144,
    118,
    115,
    111,
    110,
    91,
    89,
    86,
    64,
    38,
    587,
    9339,
    13919,
    6590,
    6821,
    7008,
    7366,
    7930,
    8137,
    8014,
    7964,
    9313,
    10812,
    9506,
    8446,
    7484,
    6757,
    6314,
    5

#### Example Gross Gamma Counts Instrument Status Packet

In [54]:
from datetime import datetime

message = {
    'instrumentStatusPacketNumber'    : 2,
    'timeInformationInstrumentStatus' :
    {
        'relativeTime'                : 1050,
        'timeStamp'                   : datetime.fromisoformat('2023-03-20T15:23:27')
    },
    'instrumentStatusInformation'     :
    {
        'instrumentPower'             : ('instrumentPowerStatus', True),
        'instrumentHealth'            : True,
        'instrumentSelfCalibration'   : True
    },
    'detectorsListInformationStatus'  :
    [
        {
            'detectorID'              : 0,
            'detectorHealth'          : True
        }
    ]
}


# message = {
#     'instrumentStatusPacketNumber'     : 2,
#     'timeInformationInstrumentStatus'  : {'relativeTime'     : 1005,
#                                          'timeStamp'        : datetime.fromisoformat('2023-03-20T15:23:27')},
#     'instrumentStatusInformation'      : {'instrumentTemperature' : 23,
#                                          'instrumentPower'       : ('instrumentBatteryRemainingCharge', 70),
#                                          'instrumentHealth'      : True},
#     'detectorsListInformationStatus'   : [{'detectorID'          : 0,
#                                            'detectorVoltage'     : -1290,
#                                            'detectorTemperature' : 34
#                                           }],
#     'endInstrumentStatusPacket'       : 1111000
# }

message

{'instrumentStatusPacketNumber': 2,
 'timeInformationInstrumentStatus': {'relativeTime': 1050,
  'timeStamp': datetime.datetime(2023, 3, 20, 15, 23, 27)},
 'instrumentStatusInformation': {'instrumentPower': ('instrumentPowerStatus',
   True),
  'instrumentHealth': True,
  'instrumentSelfCalibration': True},
 'detectorsListInformationStatus': [{'detectorID': 0, 'detectorHealth': True}]}

#### Example Gross Gamma-Neutron Counts Header Packet

In [55]:
message = {
    'timeInformationHeader' :
    {
        'timeStampZero'                          : datetime.fromisoformat('2023-03-20T15:23:27'),
        'instrumentStatusPacketTransmissionRate' : 0.0167
    },
    'instrumentInformationHeader' : 
    {
        'instrumentName'                         : "Example SGC Detector",
        'instrumentDescription'                  : "Sample Spectroscopic Gamma Detector",
        'instrumentManufacturer'                 : "ANSI",
        'instrumentModel'                        : "SGC 1.00",
        'instrumentSerialNumber'                 : "SGC-00000001",
        'numberOfDetectors'                      : 1
    },
    'detectorsListHeader' :
    [
        {
            'detectorID'                         : 0,
            'detectorType'                       : "spectroscopic",
            'detectorKind'                       : "cLYC",
            'spectrumCompression'                : False,
            'backgroundSubtraction'              : False,
            'numberOfMCAChannels'                : 1000
        }
    ]
}

# message = {
#     'timeInformationHeader'       : 
#         {
#          'timeStampZero'                       : datetime.fromisoformat('2023-03-20T15:23:27'),
#          'radiationDataPacketTransmissionRate' : 2.0
#         },
#     'instrumentInformationHeader' : 
#         {
#         'instrumentName'         : "GN1",
#         'instrumentDescription'  : 'Sample Gross Gamma Counts Detector',
#         'instrumentManufacturer' : 'IEEE',
#         'instrumentModel'        : 'Basic Gamma-Neutron Instrument 1.00',
#         'numberOfDetectors'      : 2,
#         'instrumentSerialNumber' : 'AAA-0001'
#         },
#     'detectorsListHeader' :
#         [
#             {
#             'detectorID'            : 0,
#             'detectorType'          : 'grossGamma',
#             'detectorKind'          : 'naI',
#             'detectorManufacturer'  : 'IEEE',
#             'detectorModel'         : 'Basic Gamma 1.00',
#             'exposureCoeff'         : 7.5,
#             'backgroundSubtraction' : True,
#             'numberOfMCAChannels'   : 1,
#             'calibration' :
#                 {
#                 'calibrationTime' : datetime.fromisoformat('2023-03-20T09:23:27'),
#                 }
#             },
#             {
#             'detectorID'            : 1,
#             'detectorType'          : 'grossNeutron',
#             'detectorKind'          : 'he3',
#             'detectorDescription'   : 'Sample Gross Neutron Counts Detector',
#             'detectorManufacturer'  : 'IEEE',
#             'detectorModel'         : 'Basic Gamma 1.00',
#             'detectorSerialNumber'  : 'N-00000001',
#             'doseConversionCoeff'   : 20,
#             'numberOfMCAChannels'   : 1,
#             },
#         ],
# #     'endHeaderPacket' : 10010
# }

message

{'timeInformationHeader': {'timeStampZero': datetime.datetime(2023, 3, 20, 15, 23, 27),
  'instrumentStatusPacketTransmissionRate': 0.0167},
 'instrumentInformationHeader': {'instrumentName': 'Example SGC Detector',
  'instrumentDescription': 'Sample Spectroscopic Gamma Detector',
  'instrumentManufacturer': 'ANSI',
  'instrumentModel': 'SGC 1.00',
  'instrumentSerialNumber': 'SGC-00000001',
  'numberOfDetectors': 1},
 'detectorsListHeader': [{'detectorID': 0,
   'detectorType': 'spectroscopic',
   'detectorKind': 'cLYC',
   'spectrumCompression': False,
   'backgroundSubtraction': False,
   'numberOfMCAChannels': 1000}]}

## Example Spectroscopic Packets

In [115]:
### Header ###
# message = {
#     'timeInformationHeader' :
#         {
#             'timeStampZero' : datetime.fromisoformat('2023-03-20T15:23:27'),
#             'radiationDataPacketTransmissionRate' : 0.01
#         },
#         'instrumentInformationHeader' :
#         {
#             'instrumentName' : "Example SG-RIID Detector",
#             'instrumentDescription' : "Sample Spectroscopic Gamma Detector with Radioisotope Identification",
#             'instrumentManufacturer' : "Nuclear Detection Company",
#             'instrumentModel' : "GS450",
#             'instrumentSerialNumber' : "SG_RIID-00000001",
#             'instrumentFirmware' : "RIID-PROM",
#             'instrumentFirmwareVersion' : "1.1.1"
#         },
#         'detectorsListHeader' : [
#         {
#             'detectorID' : 0,
#             'detectorType' : "spectroscopicRIID",
#             'detectorKind' : "naI",
#             'detectorDescription' : "Sodium Iodide with RIID",
#             'detectorManufacturer' : "Nuclear Detection Company",
#             'detectorSerialNumber' : "NaI_RIID-00000001",
#             'detectorFirmware' : "NaI-PROM",
#             'detectorFirmwareVersion' : "1.1.1",
#             'exposureCoeff' : 1.0,
#             'spectrumCompression' : True,
#             'numberOfMCAChannels' : 256,
#             'calibration' :
#             {
#                 'calibrationTemperaturesList' : [-20, -10, 0, 10, 20, 30, 40, 50],
#                 'calibrationEnergiesList' : [122, 239, 352, 609, 968, 1460, 1764, 2204, 2614],
#                 'calibrationChannelsList' : [[100, 320, 484, 892, 1402, 1900, 2271, 2910, 3305],
#                                              [106, 326, 491, 899, 1411, 1912, 2282, 2922, 3317],
#                                              [109, 330, 495, 906, 1419, 1925, 2291, 2935, 3321],
#                                              [112, 334, 499, 911, 1423, 1930, 2295, 2939, 3326],
#                                              [115, 338, 504, 915, 1428, 1935, 2300, 2945, 3332],
#                                              [119, 343, 508, 919, 1433, 1941, 2306, 2951, 3339],
#                                              [122, 347, 512, 925, 1438, 1947, 2313, 2958, 3345],
#                                              [125, 351, 518, 934, 1445, 1953, 2320, 2966, 3353]],
#                 'calibrationTime' : datetime.fromisoformat('2023-03-20T09:23:27'),
#                 'energyBinsStructure' : ('energyBinsList', [21.62881741,  22.26863737,  22.90853845,  23.54852065,
#                                                             24.18858398,  24.82872843,  25.468954  ,  26.1092607 ,
#                                                             26.74964852,  27.39011747,  28.03066753,  28.67129873,
#                                                             29.31201104,  29.95280448,  30.59367904,  31.23463472,
#                                                             31.87567153,  32.51678946,  33.15798852,  33.7992687 ,
#                                                             34.44063   ,  35.08207243,  35.72359598,  36.36520065,
#                                                             37.00688644,  37.64865336,  38.29050141,  38.93243057,
#                                                             39.57444086,  40.21653227,  40.85870481,  41.50095847,
#                                                             42.14329325,  42.78570916,  43.42820619,  44.07078435,
#                                                             44.71344362,  45.35618402,  45.99900555,  46.64190819,
#                                                             47.28489197,  47.92795686,  48.57110288,  49.21433002,
#                                                             49.85763828,  50.50102767,  51.14449818,  51.78804982,
#                                                             52.43168258,  53.07539646,  53.71919146,  54.36306759,
#                                                             55.00702485,  55.65106322,  56.29518272,  56.93938334,
#                                                             57.58366509,  58.22802796,  58.87247195,  59.51699707,
#                                                             60.16160331,  60.80629067,  61.45105916,  62.09590877,
#                                                             62.7408395 ,  63.38585136,  64.03094434,  64.67611844,
#                                                             65.32137367,  65.96671002,  66.6121275 ,  67.25762609,
#                                                             67.90320581,  68.54886666,  69.19460863,  69.84043172,
#                                                             70.48633593,  71.13232127,  71.77838773,  72.42453532,
#                                                             73.07076403,  73.71707386,  74.36346481,  75.00993689,
#                                                             75.6564901 ,  76.30312442,  76.94983987,  77.59663644,
#                                                             78.24351414,  78.89047296,  79.5375129 ,  80.18463397,
#                                                             80.83183616,  81.47911947,  82.12648391,  82.77392947,
#                                                             83.42145615,  84.06906396,  84.71675289,  85.36452294,
#                                                             86.01237412,  86.66030642,  87.30831985,  87.9564144 ,
#                                                             88.60459007,  89.25284686,  89.90118478,  90.54960382,
#                                                             91.19810399,  91.84668527,  92.49534769,  93.14409122,
#                                                             93.79291588,  94.44182166,  95.09080857,  95.7398766 ,
#                                                             96.38902575,  97.03825603,  97.68756743,  98.33695995,
#                                                             98.9864336 ,  99.63598836, 100.28562426, 100.93534127,
#                                                            101.58513941, 102.23501868, 102.88497907, 103.53502058,
#                                                            104.18514321, 104.83534697, 105.48563185, 106.13599785,
#                                                            106.78644498, 107.43697323, 108.08758261, 108.7382731 ,
#                                                            109.38904472, 110.03989747, 110.69083134, 111.34184633,
#                                                            111.99294244, 112.64411968, 113.29537805, 113.94671753,
#                                                            114.59813814, 115.24963987, 115.90122273, 116.55288671,
#                                                            117.20463181, 117.85645804, 118.50836539, 119.16035386,
#                                                            119.81242346, 120.46457418, 121.11680602, 121.76911899,
#                                                            122.42151308, 123.07398829, 123.72654463, 124.37918209,
#                                                            125.03190067, 125.68470038, 126.33758121, 126.99054316,
#                                                            127.64358624, 128.29671044, 128.94991577, 129.60320222,
#                                                            130.25656979, 130.91001848, 131.5635483 , 132.21715924,
#                                                            132.87085131, 133.5246245 , 134.17847881, 134.83241425,
#                                                            135.4864308 , 136.14052849, 136.79470729, 137.44896722,
#                                                            138.10330828, 138.75773045, 139.41223375, 140.06681817,
#                                                            140.72148372, 141.37623039, 142.03105818, 142.6859671 ,
#                                                            143.34095714, 143.99602831, 144.65118059, 145.306414  ,
#                                                            145.96172854, 146.6171242 , 147.27260098, 147.92815888,
#                                                            148.58379791, 149.23951806, 149.89531934, 150.55120173,
#                                                            151.20716526, 151.8632099 , 152.51933567, 153.17554256,
#                                                            153.83183058, 154.48819972, 155.14464998, 155.80118136,
#                                                            156.45779387, 157.11448751, 157.77126226, 158.42811814,
#                                                            159.08505514, 159.74207327, 160.39917252, 161.05635289,
#                                                            161.71361439, 162.37095701, 163.02838076, 163.68588562,
#                                                            164.34347161, 165.00113873, 165.65888696, 166.31671633,
#                                                            166.97462681, 167.63261842, 168.29069115, 168.948845  ,
#                                                            169.60707998, 170.26539608, 170.92379331, 171.58227166,
#                                                            172.24083113, 172.89947172, 173.55819344, 174.21699628,
#                                                            174.87588025, 175.53484534, 176.19389155, 176.85301889,
#                                                            177.51222735, 178.17151693, 178.83088764, 179.49033947,
#                                                            180.14987242, 180.8094865 , 181.4691817 , 182.12895802,
#                                                            182.78881547, 183.44875404, 184.10877373, 184.76887455,
#                                                            185.42905649, 186.08931955, 186.74966374, 187.41008905]
#                     )
#                     }
#             }
#         ],
# #     'endHeaderPacket' : 11101101
# }


### Instrument Status ###
message = {
        'instrumentStatusPacketNumber' : 1,
        'timeInformationInstrumentStatus' :
        {
            'relativeTime' : 1200,
        },
        'instrumentStatusInformation' :
        {
            'instrumentTemperature' : 23,
            'instrumentPower' : ('instrumentBatteryVoltage', 12.6),
            'instrumentHealth' : True,
            'instrumentSelfCalibration' : True
        },
        'detectorsListInformationStatus' : [
        {
            'detectorID' : 0,
            'detectorVoltage' : -1300.0,
            'detectorGain' : "5204",
            'detectorHealth' : True
        },
#         'endInstrumentStatusPacket' : 11001110
        ]
    }

### Radiation Data ###
# message = {
#     'radiationDataPacketNumber' : 1,
#     'timeInformationRadiationData' :
#     {
#       'relativeTime' : 2400,
#       'accumulationTime' : 1000,
#       'timeStamp' : datetime.fromisoformat('2023-03-20T15:23:29'),
#       'instrumentLocation' : {48.8582602,2.2944991}
#     },
#     'detectorsListRadiationData' : [
#     {
#       'counts' : 273058,
#       'channelSpectrum' : [70, 103, 149, 265, 537, 1340, 2805, 4657, 5244,
#                              4255, 2561, 1859, 1584, 1640, 1692, 1674, 1686,
#                              1675, 1717, 1743, 1708, 1739, 1782, 1779, 1720,
#                              1825, 1913, 1908, 1907, 1970, 2044, 2009, 2030,
#                              2031, 2027, 2049, 1998, 2028, 1997, 1991, 1980,
#                              1968, 2025, 1991, 2091, 2256, 2437, 2529, 2774,
#                              2751, 2699, 2588, 2466, 2389, 2367, 2284, 2191,
#                              2186, 1978, 2091, 1903, 2004, 1782, 1795, 1735,
#                              1689, 1688, 1645, 1663, 1603, 1475, 1573, 1496,
#                              1509, 1507, 1459, 1454, 1413, 1386, 1427, 1406,
#                              1322, 1342, 1293, 1325, 1320, 1285, 1313, 1250,
#                              1384, 1290, 1322, 1309, 1269, 1297, 1260, 1278,
#                              1325, 1226, 1270, 1274, 1312, 1313, 1267, 1312,
#                              1378, 1392, 1351, 1392, 1393, 1420, 1372, 1362,
#                              1454, 1450, 1437, 1331, 1255, 1185, 1064, 875,
#                              762, 663, 562, 525, 440, 437, 391, 365, 341, 307,
#                              293, 288, 215, 187, 210, 200, 169, 184, 175, 156,
#                              162, 146, 151, 121, 136, 103, 117, 116, 124, 147,
#                              123, 115, 180, 198, 294, 402, 620, 979, 1365, 2107,
#                              3058, 4206, 5325, 6299, 6559, 6459, 5593, 4257,
#                              2912, 1788, 945, 492, 251, 125, 67, 60, 62, 49, 43,
#                              63, 48, 32, 48, 51, 28, 41, 41, 31, 45, 40, 41, 40,
#                              40, 44, 47, 44, 47, 49, 49, 40, 41, 40, 39, 50, 49,
#                              37, 33, 32, 31, 33, 39, 44, 35, 40, 25, 26, 26, 32,
#                              34, 25, 34, 26, 30, 34, 22, 19, 36, 27, 27, 29, 27,
#                              31, 18, 23, 19, 21, 21, 22, 25, 30, 19, 19, 18, 21,
#                              11, 15, 17, 20, 18],
#       'derivedData' :
#       {
#         'countRate' : 2744.299263,
#         'exposureRate' : 2744.299263,
#         'energyBinsCounts' : {70., 103., 149., 265., 537., 1340., 2805., 4657., 5244,
#                              4255., 2561., 1859., 1584., 1640., 1692., 1674., 1686,
#                              1675., 1717., 1743., 1708., 1739., 1782., 1779., 1720,
#                              1825., 1913., 1908., 1907., 1970., 2044., 2009., 2030,
#                              2031., 2027., 2049., 1998., 2028., 1997., 1991., 1980,
#                              1968., 2025., 1991., 2091., 2256., 2437., 2529., 2774,
#                              2751., 2699., 2588., 2466., 2389., 2367., 2284., 2191,
#                              2186., 1978., 2091., 1903., 2004., 1782., 1795., 1735,
#                              1689., 1688., 1645., 1663., 1603., 1475., 1573., 1496,
#                              1509., 1507., 1459., 1454., 1413., 1386., 1427., 1406,
#                              1322., 1342., 1293., 1325., 1320., 1285., 1313., 1250,
#                              1384., 1290., 1322., 1309., 1269., 1297., 1260., 1278,
#                              1325., 1226., 1270., 1274., 1312., 1313., 1267., 1312,
#                              1378., 1392., 1351., 1392., 1393., 1420., 1372., 1362,
#                              1454., 1450., 1437., 1331., 1255., 1185., 1064., 875,
#                              762., 663., 562., 525., 440., 437., 391., 365., 341., 307,
#                              293., 288., 215., 187., 210., 200., 169., 184., 175., 156,
#                              162., 146., 151., 121., 136., 103., 117., 116., 124., 147,
#                              123., 115., 180., 198., 294., 402., 620., 979., 1365., 2107,
#                              3058., 4206., 5325., 6299., 6559., 6459., 5593., 4257,
#                              2912., 1788., 945., 492., 251., 125., 67., 60., 62., 49., 43,
#                              63., 48., 32., 48., 51., 28., 41., 41., 31., 45., 40., 41., 40,
#                              40., 44., 47., 44., 47., 49., 49., 40., 41., 40., 39., 50., 49,
#                              37., 33., 32., 31., 33., 39., 44., 35., 40., 25., 26., 26., 32,
#                              34., 25., 34., 26., 30., 34., 22., 19., 36., 27., 27., 29., 27,
#                              31., 18., 23., 19., 21., 21., 22., 25., 30., 19., 19., 18., 21,
#                              11., 15., 17., 20., 18.},
#         'alarm1' : True,
#         'deadTime' : 6.375924139,
#         'liveTime' : 255648.0,
#     }
#     }
#     ],
# #     'endRadiationDataPacket' : 10111110
# }

In [109]:
message

{'instrumentStatusPacketNumber': 1,
 'timeInformationInstrumentStatus': {'relativeTime': 1200},
 'instrumentStatusInformation': {'instrumentTemperature': 23,
  'instrumentPower': ('instrumentBatteryVoltage', 12.6),
  'instrumentHealth': True,
  'instrumentSelfCalibration': True},
 'detectorsListInformationStatus': [{'detectorID': 0,
   'detectorVoltage': -1300.0,
   'detectorGain': '5204',
   'detectorHealth': True}]}

In [110]:
# encoded = spec.encode('Header', message)
encoded = spec.encode('InstrumentStatus', message)
# encoded = spec.encode('RadiationData', message)
# encoded = spec.encode('Footer', message)

In [111]:
# print(encoded)

In [112]:
# bin(int.from_bytes(encoded, byteorder="big")).strip('0b')

In [113]:
print("Length of encoded message:", len(encoded), "bytes")

Length of encoded message: 57 bytes


In [114]:
bin(sum(encoded))[-8:]

'11001110'